In [1]:
import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [2]:
import pandas as pd
import numpy as np
from nltk.corpus import wordnet as wn
import gensim 
from gensim.models import Word2Vec 
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

In [4]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')

In [5]:
from nltk.corpus import stopwords
breakers = {'with', 'without', 'comma', 'and', 'or', 'by', 'in', 'due', 'to', 'of', 'causing'}
stop_words= (set(stopwords.words("english")) | {'nan', 'unspecified', 'stated', 'other', 'state'}) - breakers

In [6]:
from nltk.stem.wordnet import WordNetLemmatizer
lem = WordNetLemmatizer()

In [7]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()

In [3]:
df_train = np.asarray(pd.read_csv("data/diabetes/diab_train.csv", skiprows=1, header=None))
df_val = np.asarray(pd.read_csv("data/diabetes/diab_validation.csv", skiprows=1, header=None))
df_test = np.asarray(pd.read_csv("data/diabetes/diab_test.csv", skiprows=1, header=None))

In [8]:
X = np.hstack([df_train[:,:-4], df_train[:,-3:]])
X_val = np.hstack([df_val[:,:-4], df_val[:,-3:]])
X_test = np.hstack([df_test[:,:-4], df_test[:,-3:]])
Y = df_train[:,-4]
Y_val = df_val[:,-4]
Y_test = df_test[:,-4]

In [10]:
X_all = np.vstack([X, X_val, X_test])

In [9]:
def sanitize(doc):
    return ' '.join([lem.lemmatize(w) for w in tokenizer.tokenize(str(doc).replace(",", " comma").lower()) if w not in stop_words])

Race

In [16]:
print(set(X_all[:,1]))

{'Other', '?', 'Caucasian', 'AfricanAmerican', 'Asian', 'Hispanic'}


Gender

In [17]:
print(set(X_all[:,2]))

{'Female', 'Male'}


Age

In [18]:
print(set(X_all[:,3]))

{'[40-50)', '[90-100)', '[20-30)', '[50-60)', '[60-70)', '[10-20)', '[0-10)', '[80-90)', '[70-80)', '[30-40)'}


Weight

In [19]:
print(set(X_all[:,4]))

{'[125-150)', '[100-125)', '[25-50)', '?', '[150-175)', '[0-25)', '[75-100)', '[50-75)'}


Admission Type ID

In [20]:
print(set(X_all[:,5]))

{nan, 'Newborn', 'Not Mapped', 'Emergency', 'Urgent', 'Elective', 'Not Available'}


Discharge Disposition ID

In [63]:
print(len(set(X_all[:,6])), 'Items')
print(set(X_all[:,6]))

22 Items
{nan, 'Discharged/transferred to a nursing facility certified under Medicaid but not certified under Medicare', 'Discharged/transferred/referred to this institution for outpatient services', 'Hospice / home', 'Discharged/transferred/referred another institution for outpatient services', 'Discharged/transferred to a federal health care facility.', 'Left AMA', 'Discharged/transferred to another  type of inpatient care institution', 'Expired', 'Discharged/transferred to another short term hospital', 'Discharged/transferred to SNF', 'Hospice / medical facility', 'Not Mapped', 'Discharged/transferred to another rehab fac including rehab units of a hospital.', 'Discharged/transferred/referred to a psychiatric hospital of a psychiatric distinct part unit of a hospital', 'Discharged to home', 'Discharged/transferred to ICF', 'Discharged/transferred to home with home health service', 'Discharged/transferred to home under care of Home IV provider', 'Discharged/transferred to a long term

Admission Source ID

In [23]:
print(set(X_all[:,7]))

{nan, 'Not Available', 'Not Mapped', 'Physician Referral', 'Clinic Referral', 'Transfer from another health care facility', 'Transfer from a hospital', 'HMO Referral', 'Emergency Room', 'Transfer from a Skilled Nursing Facility (SNF)', 'Court/Law Enforcement'}


Time in Hospital

In [24]:
print(set(X_all[:,8]))

{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14}


Payer Code

In [25]:
print(set(X_all[:,9]))

{'PO', 'SP', 'CP', 'OT', 'MD', 'DM', 'BC', '?', 'MC', 'CH', 'SI', 'OG', 'UN', 'HM', 'CM', 'WC'}


Medical Speciality

In [28]:
print(sorted(list(set(X_all[:,10]))))

['?', 'Anesthesiology-Pediatric', 'Cardiology', 'Cardiology-Pediatric', 'Emergency/Trauma', 'Endocrinology', 'Family/GeneralPractice', 'Gastroenterology', 'Gynecology', 'Hematology', 'Hematology/Oncology', 'Hospitalist', 'InfectiousDiseases', 'InternalMedicine', 'Nephrology', 'Neurology', 'Obsterics&Gynecology-GynecologicOnco', 'Obstetrics', 'ObstetricsandGynecology', 'Oncology', 'Ophthalmology', 'Orthopedics', 'Orthopedics-Reconstructive', 'Osteopath', 'Otolaryngology', 'OutreachServices', 'Pathology', 'Pediatrics', 'Pediatrics-CriticalCare', 'Pediatrics-EmergencyMedicine', 'Pediatrics-Endocrinology', 'Pediatrics-Hematology-Oncology', 'Pediatrics-Pulmonology', 'PhysicalMedicineandRehabilitation', 'PhysicianNotFound', 'Podiatry', 'Psychiatry', 'Psychology', 'Pulmonology', 'Radiologist', 'Radiology', 'Surgeon', 'Surgery-Cardiovascular', 'Surgery-Cardiovascular/Thoracic', 'Surgery-Colon&Rectal', 'Surgery-General', 'Surgery-Neuro', 'Surgery-Pediatric', 'Surgery-Plastic', 'Surgery-Plasticw

Number of Lab Procedures

In [29]:
print(set(X_all[:,11]))

{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 101, 103, 104, 106, 107, 108, 109, 113, 114, 120}


Number of Procedures

In [30]:
print(set(X_all[:,12]))

{0, 1, 2, 3, 4, 5, 6}


Number of Medications

In [31]:
print(set(X_all[:,13]))

{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 65, 67, 74, 75, 81}


Number Outpatient

In [32]:
print(set(X_all[:,14]))

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 21, 27, 36}


Number Emergency

In [33]:
print(set(X_all[:,15]))

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 42, 13}


Number Inpatient

In [34]:
print(set(X_all[:,16]))

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10}


Diagnoses

In [66]:
print(set(X_all[:,17])|set(X_all[:,18])|set(X_all[:,19]))

{'655', 'V85', '323', '583', '438', '230', 'V57', '610', '314', '474', '430', '459', '537', '250.51', '510', '808', '285', '873', 'V65', '250.5', 'E935', '753', '734', '203', '752', '465', '413', '250.33', '682', '277', '485', '372', '261', '723', '490', '867', '429', '262', '527', '915', 'V14', '945', '731', '280', '494', '747', '250.12', '179', '273', '713', '709', '405', '374', '991', 'V49', 'E933', '349', '784', '117', '250.13', '253', '543', 'V71', 'E882', '656', 'E949', '692', '869', '916', '275', '435', '397', '337', '562', '398', '354', '738', 'V08', 'V46', '786', '291', '820', '861', '201', '816', '614', '846', '475', '910', '319', '905', 'E815', '715', '27', '720', '536', '793', '368', '220', 'V63', '785', '200', '795', '411', '250.6', '553', '420', '669', '299', '78', 'V25', '250.02', '571', '796', '493', 'V09', '933', 'E822', '836', '340', '518', '845', '691', '693', '512', '727', '824', '198', '535', '552', '250.91', '184', '250.53', '342', '470', '707', 'E888', '540', '35

Number of Diagnoses

In [37]:
print(set(X_all[:,20]))

{1, 2, 3, 4, 5, 6, 7, 8, 9}


Max Glucose Serum

In [38]:
print(set(X_all[:,21]))

{'>200', 'Norm', '>300', 'None'}


A1Cresult

In [39]:
print(set(X_all[:,22]))

{'>7', 'Norm', '>8', 'None'}


Metformin - Metformin Pioglitazone

In [40]:
print(set(X_all[:,23]))

{'Up', 'No', 'Steady', 'Down'}


In [61]:
drug_change = {'Down': -2, 'No': 0, 'Steady': 1, 'Up': 2}

Change

In [68]:
print(set(X_all[:,46]))

{'No', 'Ch'}


Diabetes Medication

In [69]:
print(set(X_all[:,47]))

{'Yes', 'No'}


Diagnoses Description

In [60]:
print(set(X_all[:,48]))

{nan, 'Osteoporosis, unspecified', 'Generalized nonconvulsive epilepsy, without mention of intractable epilepsy', 'Pneumonia due to mycoplasma pneumoniae', 'Acute vascular insufficiency of intestine', 'Postmyocardial infarction syndrome', 'Dissection of aorta, unspecified site', 'Paranoid personality disorder', 'Diabetes with ketoacidosis, type I [juvenile type], uncontrolled', 'Benign neoplasm of lip', 'Poisoning by adrenal cortical steroids', 'Headache', 'Care involving breathing exercises', 'Diabetes with hyperosmolarity, type I [juvenile type], not stated as uncontrolled', 'Common truncus', 'Sprain of ankle, unspecified site', 'Hypertrophy of nasal turbinates', 'Injury to bladder and urethra, without mention of open wound into cavity', 'Other and unspecified intracranial hemorrhage following injury without mention of open intracranial wound, unspecified state of consciousness', 'First-degree perineal laceration, unspecified as to episode of care or not applicable', 'Myocarditis, un